# Práctica Guiada - Técnicas para datasets desbalanceados

El problema de los datasets desbalanceados se encuentra en una amplia variedad de problemas de Machine Learning como la detección de comentarios hechos a través de bots, el diagnóstico de enfermedades o la búsqueda de transacciones fraudulentas en un sistema.

Existen dos grandes enfoques para abordar datasets que se encuentran desbalanceados:

1.  Hacer un resampling de la muestra, para entrenar al algoritmo con proporciones similares
2.  Incorporar el desbalance a la función de costos del algoritmo para que tenga incentivos a elegir los parámetros que mejor discriminan la clase minoritaria.

## Clasificación sobre datos desbalanceados

A continuación presentamos un dataset de la empresa americana Lending Club, que se dedica a proveer servicios financieros para distintos segmentos. 

A continuación vamos a utilizar información abierta del portal de la empresa para intentar predecir cuáles de los créditos terminan en default. Para más información sobre el datset pueden ingresar <a href='https://www.lendingclub.com/info/download-data.action'> aquí </a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
loans = pd.read_csv('loans.csv',low_memory=False)

## Análisis exploratorio

#### Balanceo de la clase

La clase que vamos a intentar predecir es "bad loans" que indica si el préstamo fue pagado a tiempo o no. Observamos que la clase está desbalanceada, la mayoría de los préstamos se pagana tiempo.

In [ ]:
loans['bad_loans'].value_counts(normalize=True)

#### Variables numéricas
Podemos buscar las variables numéricas relevantes observando cuáles son las que tienen mayor correlación con la clase, bad_loans.

In [ ]:
loans.corr()['bad_loans'].apply(abs).sort_values(ascending=False)

#### Variables categóricas

Para hacer el mismo análisis sobre variables categóricas, hacemos un group by por categoría para ver diferencias relevantes en la cantidad de créditos con default.

In [ ]:
loans.groupby('home_ownership')['bad_loans'].mean().sort_values()

In [ ]:
loans.groupby('term')['bad_loans'].mean().sort_values()

In [ ]:
loans.groupby('purpose')['bad_loans'].mean().sort_values()

#### Valores faltantes

Inspeccionamos los valores faltantes

In [ ]:
loans.isnull().sum()

In [ ]:
# Notamos que la variable payment_inc_ratio tiene sólo 4 valores nulos y mustra una correlación importante con la clase.
# Además sabemos que la relación cuota/ingreso es importante a al hora de evaluar la capacidad de repago. 
# Eliminamos, entonces, los valores con payment_inc_ratio desconocido.
loans = loans[loans.payment_inc_ratio.notnull()].copy()

In [ ]:
loans.sample(10)

## Modelo Predictivo: regresión logística

Vamos a utilizar Regresión Logística para predecir la clase bad_loans. 

In [ ]:
# Generamos las variables dummies para los datos categóricos.
loans_enconded = pd.get_dummies(loans)

In [ ]:
training_features, test_features, \
training_target, test_target, = train_test_split(loans_enconded.drop(['bad_loans'], axis=1),
                                               loans_enconded['bad_loans'],
                                               test_size = .1,
                                               random_state=12)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report


In [ ]:
model = LogisticRegression(C=1e10)
model.fit(training_features,training_target)

In [ ]:
y_pred = model.predict(test_features)

In [ ]:
roc_auc_score(test_target,y_pred)

In [ ]:
print(classification_report(test_target,y_pred))

Cuando evaluamos el modelo sobre datos no observados, el área debajo de la curva es muy cercana a 0.5. Especificamente, el recall a la hora de encontrar los "malos créditos" es  muy malo. Uno de los problemas que tenemos para hacer un buen modelo es el desbalanceo de los datos. 

Vamos a intentar dos posibles soluciones.

## Oversampling

Para aumentar la representación de la clase minoritaria vamos a hacer un oversampling utilizando el algoritmo SMOTE (Synthetic Minority Oversample) del paquete imblearn.

Este algoritmo genera nuevos datos utilizando la técnica de los K vecinos más cercanos.
Para generar un nuevo punto:

1. Se elige un punto al azar de la clase minoritaria y sus K vecinos más cercanos.
2. Se elige al azar uno de esos vecinos.
3. Se calcula el vector entre el punto seleccionado y el vecino seleccionado al azar y se lo multiplica por un número aleatorio entre 0 y 1.
4. El punto aleatorio dentro del vector es el nuevo dato para el oversampling.


In [ ]:
sm = SMOTE(random_state=12)
x_train_res, y_train_res = sm.fit_sample(training_features, training_target)

In [ ]:
model.fit(x_train_res,y_train_res)

In [ ]:
y_pred = model.predict(test_features)

In [ ]:
roc_auc_score(test_target,y_pred)

In [ ]:
print(classification_report(test_target,y_pred))

# Class Weights

La otra técnica que podemos utilizar para corregir el desbalance de los datos es incorporar en la función de costos del algoritmo un mayor peso para los errores de entrenamiento cometidos sobre los puntos de la clase minoritaria.

<strong> Ejercicio: Implementar la Regresión Logística con un parámetro que cambie la ponderación de las clases. <br />
¿Cómo es la performance del nuevo modelo con respecto a la regresión logística original? <br />
¿Tiene sentido aplicar esta corrección junto con el algoritmo SMOTE?
<strong />

In [ ]:
model = LogisticRegression(C=1e10, class_weight = 'balanced')

In [ ]:
model.fit(training_features,training_target)

In [ ]:
y_pred = model.predict(test_features)

In [ ]:
roc_auc_score(test_target,y_pred)

In [ ]:
print(classification_report(test_target,y_pred))